### Import Libraries and Configurations


In [103]:

import pandas as pd
pd.options.display.max_columns = None

import altair as alt
alt.renderers.enable('mimetype')

from vega_datasets import data
import vega

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")


### Read data, join multiple datasets, general statistics before cleaning 

In [104]:
# Reading in the main dataframe (cause of death)
df=pd.read_csv('cause_of_deaths.csv')
df.head()

,Country/Territory,Code,Year,Meningitis,Alzheimer's Disease and Other Dementias,Parkinson's Disease,Nutritional Deficiencies,Malaria,Drowning,Interpersonal Violence,Maternal Disorders,HIV/AIDS,Drug Use Disorders,Tuberculosis,Cardiovascular Diseases,Lower Respiratory Infections,Neonatal Disorders,Alcohol Use Disorders,Self-harm,Exposure to Forces of Nature,Diarrheal Diseases,Environmental Heat and Cold Exposure,Neoplasms,Conflict and Terrorism,Diabetes Mellitus,Chronic Kidney Disease,Poisonings,Protein-Energy Malnutrition,Road Injuries,Chronic Respiratory Diseases,Cirrhosis and Other Chronic Liver Diseases,Digestive Diseases,"Fire, Heat, and Hot Substances",Acute Hepatitis
0,Afghanistan,AFG,1990,2159,1116,371,2087,93,1370,1538,2655,34,93,4661,44899,23741,15612,72,696,0,4235,175,11580,1490,2108,3709,338,2054,4154,5945,2673,5005,323,2985
1,Afghanistan,AFG,1991,2218,1136,374,2153,189,1391,2001,2885,41,102,4743,45492,24504,17128,75,751,1347,4927,113,11796,3370,2120,3724,351,2119,4472,6050,2728,5120,332,3092
2,Afghanistan,AFG,1992,2475,1162,378,2441,239,1514,2299,3315,48,118,4976,46557,27404,20060,80,855,614,6123,38,12218,4344,2153,3776,386,2404,5106,6223,2830,5335,360,3325
3,Afghanistan,AFG,1993,2812,1187,384,2837,108,1687,2589,3671,56,132,5254,47951,31116,22335,85,943,225,8174,41,12634,4096,2195,3862,425,2797,5681,6445,2943,5568,396,3601
4,Afghanistan,AFG,1994,3027,1211,391,3081,211,1809,2849,3863,63,142,5470,49308,33390,23288,88,993,160,8215,44,12914,8959,2231,3932,451,3038,6001,6664,3027,5739,420,3816


In [105]:
# Basic Stats about the cause of death dataframe
print("Shape:", df.shape)
print("Unique Countries:",df['Country/Territory'].nunique())
print("Null Elements: ", df.isna().sum().sum())
print("Earliest Year: ", df['Year'].min())
print("Latest Year: ", df['Year'].max())
print("Number of Causes of deaths: ", df.shape[1]-3)

Shape: (6120, 34)
Unique Countries: 204
Null Elements:  0
Earliest Year:  1990
Latest Year:  2019
Number of Causes of deaths:  31


In [106]:
# Read in UID dataframe which contains each country's Unique Identifer and Lat and Long 
country_data = pd.read_csv("UID.csv")
country_data=country_data[country_data['Province_State'].isnull()][['UID','Country_Region','Lat','Long_']]
country_data.loc[country_data['Country_Region'] == "US",'Country_Region'] = "United States"
country_data.head()

,UID,Country_Region,Lat,Long_
0,4,Afghanistan,33.93911,67.709953
1,8,Albania,41.15330,20.168300
2,10,Antarctica,-71.94990,23.347000
3,12,Algeria,28.03390,1.659600
4,20,Andorra,42.50630,1.521800


In [107]:
# Merge cause_of_deaths dataframe with UID dataframe 
df=pd.merge(left=df,right=country_data,left_on="Country/Territory",right_on="Country_Region")


In [108]:

# Data cleaning on the dataframe by make the column names more consistent
df.columns = list(map(lambda x: x.replace(' ', '_').replace('/','_').replace("'","").replace(",",""),
                                 df.columns))
df.drop(columns=['Country_Region',],inplace=True)
df.rename(columns={'Long_':'Long'},inplace=True)
df.head()                                 


,Country_Territory,Code,Year,Meningitis,Alzheimers_Disease_and_Other_Dementias,Parkinsons_Disease,Nutritional_Deficiencies,Malaria,Drowning,Interpersonal_Violence,Maternal_Disorders,HIV_AIDS,Drug_Use_Disorders,Tuberculosis,Cardiovascular_Diseases,Lower_Respiratory_Infections,Neonatal_Disorders,Alcohol_Use_Disorders,Self-harm,Exposure_to_Forces_of_Nature,Diarrheal_Diseases,Environmental_Heat_and_Cold_Exposure,Neoplasms,Conflict_and_Terrorism,Diabetes_Mellitus,Chronic_Kidney_Disease,Poisonings,Protein-Energy_Malnutrition,Road_Injuries,Chronic_Respiratory_Diseases,Cirrhosis_and_Other_Chronic_Liver_Diseases,Digestive_Diseases,Fire_Heat_and_Hot_Substances,Acute_Hepatitis,UID,Lat,Long
0,Afghanistan,AFG,1990,2159,1116,371,2087,93,1370,1538,2655,34,93,4661,44899,23741,15612,72,696,0,4235,175,11580,1490,2108,3709,338,2054,4154,5945,2673,5005,323,2985,4,33.93911,67.709953
1,Afghanistan,AFG,1991,2218,1136,374,2153,189,1391,2001,2885,41,102,4743,45492,24504,17128,75,751,1347,4927,113,11796,3370,2120,3724,351,2119,4472,6050,2728,5120,332,3092,4,33.93911,67.709953
2,Afghanistan,AFG,1992,2475,1162,378,2441,239,1514,2299,3315,48,118,4976,46557,27404,20060,80,855,614,6123,38,12218,4344,2153,3776,386,2404,5106,6223,2830,5335,360,3325,4,33.93911,67.709953
3,Afghanistan,AFG,1993,2812,1187,384,2837,108,1687,2589,3671,56,132,5254,47951,31116,22335,85,943,225,8174,41,12634,4096,2195,3862,425,2797,5681,6445,2943,5568,396,3601,4,33.93911,67.709953
4,Afghanistan,AFG,1994,3027,1211,391,3081,211,1809,2849,3863,63,142,5470,49308,33390,23288,88,993,160,8215,44,12914,8959,2231,3932,451,3038,6001,6664,3027,5739,420,3816,4,33.93911,67.709953


In [109]:
# After merging, we get 184 that has country UID, down from 204
print("Unique Countries:",df['Country_Territory'].nunique())

Unique Countries: 184


### Question 1: How are the terrorist attacks in the US starting the year 2000 affects the death counts in the US and Afghanistan?


In [110]:

USA_Terrorism =df.loc[(df.Country_Territory=="United States") & (df.Year>=2000)][['Year','Conflict_and_Terrorism']].sort_values('Year').reset_index(drop=True)
USA_Terrorism["USA_Difference"] = USA_Terrorism['Conflict_and_Terrorism'].diff()
USA_Terrorism.rename(columns={'Conflict_and_Terrorism':'USA_Conflict_and_Terrorism'},inplace=True)

Afghanistan=df.loc[(df.Country_Territory=="Afghanistan") & (df.Year>=2000)][['Year','Conflict_and_Terrorism']].sort_values('Year').reset_index(drop=True)
Afghanistan["Afghanistan_Difference"] = Afghanistan['Conflict_and_Terrorism'].diff()
Afghanistan.rename(columns={'Conflict_and_Terrorism':'Afghanistan_Conflict_and_Terrorism'},inplace=True)


In [111]:
Terrorism=pd.merge(left=USA_Terrorism,right=Afghanistan,left_on="Year",right_on="Year")
Terrorism

,Year,USA_Conflict_and_Terrorism,USA_Difference,Afghanistan_Conflict_and_Terrorism,Afghanistan_Difference
0,2000,41,NaN,5429,NaN
1,2001,4279,4238.0,5689,260.0
2,2002,66,-4213.0,977,-4712.0
3,2003,538,472.0,864,-113.0
4,2004,927,389.0,1049,185.0
5,2005,969,42.0,2114,1065.0
6,2006,948,-21.0,5677,3563.0
7,2007,1031,83.0,8220,2543.0
8,2008,487,-544.0,6895,-1325.0
9,2009,502,15.0,7617,722.0


In [112]:
def display_map(year,column_name):
    df1 = df.loc[df.Year==year].reset_index()
    source = alt.topo_feature(data.world_110m.url, "countries")
    background = alt.Chart(source).mark_geoshape(fill="white").interactive()

    color=column_name+":N"
    tooltip=column_name+":Q"

    foreground = (
        alt.Chart(source).mark_geoshape(
            stroke="black", strokeWidth=0.15
        ).encode(
            color=alt.Color(
                color, scale=alt.Scale(scheme="lightgreyred"), legend=None
                #legend=alt.Legend(type='gradient',title='Legend',gradientLength=100,direction="horizontal", orient='bottom'),
            ),
            tooltip=[
                alt.Tooltip("Country_Territory:N", title="Country"),
                alt.Tooltip(tooltip, title=column_name),
            ],
        ).transform_lookup(
            lookup="id",
            from_=alt.LookupData(df1, "UID", [column_name, "Country_Territory"]),
        ).interactive()
    )

    chart = (
        (background + foreground)
        .configure_view(strokeWidth=0)
        .properties(title=str(year)+ " Deaths Due to " + column_name.replace('_',' ') + " World Map",width=700, height=400)
        .project("naturalEarth1")
    )

    return chart

In [113]:
display_map(2000,'Conflict_and_Terrorism')

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [114]:
display_map(2001,'Conflict_and_Terrorism')


<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [115]:
display_map(2002,'Conflict_and_Terrorism')


<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


### What diseases in the United States have greater rate than the population growth rate?

In [121]:
#join with US
us_population = pd.read_csv("US_population.csv")
us_population.head()

,Year,Population
0,1990,249623000
1,1991,252981000
2,1992,256514000
3,1993,259919000
4,1994,263126000


In [122]:
US = df.loc[df.Country_Territory=="United States"].head(50).sort_values(by=['Year'])
US = pd.merge(left=df,right=us_population,on="Year")

In [120]:
df

,Country_Territory,Code,Year,Meningitis,Alzheimers_Disease_and_Other_Dementias,Parkinsons_Disease,Nutritional_Deficiencies,Malaria,Drowning,Interpersonal_Violence,Maternal_Disorders,HIV_AIDS,Drug_Use_Disorders,Tuberculosis,Cardiovascular_Diseases,Lower_Respiratory_Infections,Neonatal_Disorders,Alcohol_Use_Disorders,Self-harm,Exposure_to_Forces_of_Nature,Diarrheal_Diseases,Environmental_Heat_and_Cold_Exposure,Neoplasms,Conflict_and_Terrorism,Diabetes_Mellitus,Chronic_Kidney_Disease,Poisonings,Protein-Energy_Malnutrition,Road_Injuries,Chronic_Respiratory_Diseases,Cirrhosis_and_Other_Chronic_Liver_Diseases,Digestive_Diseases,Fire_Heat_and_Hot_Substances,Acute_Hepatitis,UID,Lat,Long
0,Afghanistan,AFG,1990,2159,1116,371,2087,93,1370,1538,2655,34,93,4661,44899,23741,15612,72,696,0,4235,175,11580,1490,2108,3709,338,2054,4154,5945,2673,5005,323,2985,4,33.939110,67.709953
1,Afghanistan,AFG,1991,2218,1136,374,2153,189,1391,2001,2885,41,102,4743,45492,24504,17128,75,751,1347,4927,113,11796,3370,2120,3724,351,2119,4472,6050,2728,5120,332,3092,4,33.939110,67.709953
2,Afghanistan,AFG,1992,2475,1162,378,2441,239,1514,2299,3315,48,118,4976,46557,27404,20060,80,855,614,6123,38,12218,4344,2153,3776,386,2404,5106,6223,2830,5335,360,3325,4,33.939110,67.709953
3,Afghanistan,AFG,1993,2812,1187,384,2837,108,1687,2589,3671,56,132,5254,47951,31116,22335,85,943,225,8174,41,12634,4096,2195,3862,425,2797,5681,6445,2943,5568,396,3601,4,33.939110,67.709953
4,Afghanistan,AFG,1994,3027,1211,391,3081,211,1809,2849,3863,63,142,5470,49308,33390,23288,88,993,160,8215,44,12914,8959,2231,3932,451,3038,6001,6664,3027,5739,420,3816,4,33.939110,67.709953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5515,Zimbabwe,ZWE,2015,1439,754,215,3019,2518,770,1302,1355,29162,104,11214,16649,12974,9278,48,2235,16,5102,37,11161,13,3176,2108,381,2990,2373,2751,1956,4202,632,146,716,-19.015438,29.154857
5516,Zimbabwe,ZWE,2016,1457,767,219,3056,2050,801,1342,1338,27141,110,10998,16937,13024,9065,49,2296,31,5002,37,11465,6,3259,2160,393,3027,2436,2788,1962,4264,648,146,716,-19.015438,29.154857
5517,Zimbabwe,ZWE,2017,1460,781,223,2990,2116,818,1363,1312,24846,115,10762,17187,12961,8901,50,2338,251,4948,37,11744,5,3313,2196,398,2962,2473,2818,2007,4342,654,144,716,-19.015438,29.154857
5518,Zimbabwe,ZWE,2018,1450,795,227,2918,2088,825,1396,1294,22106,121,10545,17460,12860,8697,51,2372,0,4745,37,12038,9,3381,2240,400,2890,2509,2849,2030,4377,657,139,716,-19.015438,29.154857


In [118]:
col_names = US.columns.tolist()
unwanted = ['Code','Year',"Country_Territory","UID","Lat",'Long']
col_names = [ele for ele in col_names if ele not in unwanted]
for name in col_names:
    name_perchange = name+"_PerChange"
    US[name_perchange] = US[name].pct_change()*100
    US=US.drop([name],axis=1)
US = US.drop(['Code',"Country_Territory","UID","Lat",'Long'],axis=1)

In [119]:
US

,Year,Meningitis_PerChange,Alzheimers_Disease_and_Other_Dementias_PerChange,Parkinsons_Disease_PerChange,Nutritional_Deficiencies_PerChange,Malaria_PerChange,Drowning_PerChange,Interpersonal_Violence_PerChange,Maternal_Disorders_PerChange,HIV_AIDS_PerChange,Drug_Use_Disorders_PerChange,Tuberculosis_PerChange,Cardiovascular_Diseases_PerChange,Lower_Respiratory_Infections_PerChange,Neonatal_Disorders_PerChange,Alcohol_Use_Disorders_PerChange,Self-harm_PerChange,Exposure_to_Forces_of_Nature_PerChange,Diarrheal_Diseases_PerChange,Environmental_Heat_and_Cold_Exposure_PerChange,Neoplasms_PerChange,Conflict_and_Terrorism_PerChange,Diabetes_Mellitus_PerChange,Chronic_Kidney_Disease_PerChange,Poisonings_PerChange,Protein-Energy_Malnutrition_PerChange,Road_Injuries_PerChange,Chronic_Respiratory_Diseases_PerChange,Cirrhosis_and_Other_Chronic_Liver_Diseases_PerChange,Digestive_Diseases_PerChange,Fire_Heat_and_Hot_Substances_PerChange,Acute_Hepatitis_PerChange,Population_PerChange
0,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990,-95.044002,-67.741935,-74.932615,-98.227120,-1.000000e+02,-89.270073,-88.491547,-99.020716,-97.058824,-94.623656,-99.013087,-85.075391,-90.986058,-9.442096e+01,-86.111111,-82.040230,NaN,-98.205431,-97.142857,-77.409326,-100.000000,-95.256167,-93.502292,-96.153846,-98.296008,-90.491093,-83.078217,-87.803966,-87.052947,-90.402477,-99.865997,0.0
2,1990,683.177570,271.388889,372.043011,1124.324324,inf,721.088435,177.401130,6184.615385,4300.000000,3320.000000,1941.304348,698.209223,279.672897,1.958668e+03,650.000000,1024.000000,NaN,3726.315789,600.000000,304.090214,inf,1643.000000,1376.763485,3630.769231,1105.714286,3302.025316,338.369781,636.503067,514.351852,3490.322581,12050.000000,0.0
3,1990,-100.000000,-99.551234,-99.544419,-100.000000,-1.000000e+02,-100.000000,-99.796334,-100.000000,-97.727273,-100.000000,-100.000000,-99.878477,-99.926154,-9.999442e+01,-98.666667,-99.572954,NaN,-100.000000,-100.000000,-99.044556,-100.000000,-99.770511,-99.887609,-100.000000,-100.000000,-99.940467,-99.637188,-99.750104,-99.748807,-100.000000,-99.794239,0.0
4,1990,inf,4783.333333,4200.000000,inf,inf,inf,45700.000000,inf,23800.000000,inf,inf,18163.076923,326833.333333,1.420900e+06,9700.000000,16116.666667,NaN,inf,inf,4734.653465,inf,37875.000000,26375.000000,inf,inf,63175.000000,17362.500000,46266.666667,44710.000000,inf,64400.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5515,2019,4133.333333,33126.315789,8507.692308,4500.000000,6.660000e+04,1537.500000,364700.000000,6112.500000,9636.842105,8900.000000,1914.634146,6978.250863,4815.929204,4.737681e+03,4600.000000,4429.411765,NaN,1985.294118,1000.000000,15353.777778,NaN,8400.000000,14764.285714,4000.000000,4291.666667,9164.179104,4005.813953,9475.510204,9248.913043,4800.000000,600.000000,0.0
5516,2019,194.094488,208.252812,280.607685,72.826087,-9.115442e+01,1063.740458,-88.422880,-52.917505,245.891892,1206.666667,2161.622276,290.442205,284.248425,9.376872e+01,431.914894,251.255411,inf,17.912553,645.454545,223.729545,NaN,172.264937,106.487266,1040.243902,-81.973435,289.125181,413.013311,403.772379,263.980932,210.204082,3109.523810,0.0
5517,2019,-35.876841,-91.104830,-90.302888,5.870021,8.525424e+03,-86.175795,-35.280189,628.632479,-94.577278,-77.806122,-94.491730,-76.785573,-70.536425,1.934756e+02,-91.800000,-80.773971,-100.0,295.574163,-47.560976,-88.734409,inf,-93.824640,-87.544799,-51.871658,890.526316,-51.488428,-84.161859,-88.213394,-84.421517,37.993421,-22.403561,0.0
5518,2019,331.106472,-55.921433,-50.847458,88.019802,-6.130871e+00,-57.413998,97.073171,-52.844575,6395.677233,-77.011494,585.714286,-67.432558,30.227381,-4.810347e+01,67.073171,-10.384615,inf,9.222861,95.348837,-22.805772,-99.992538,41.763085,-27.840060,-31.333333,94.261424,-80.498421,-58.382712,64.860014,33.852778,-61.144219